In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
fin = pd.read_csv('/content/financials.csv')
fin.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,1.387211e+11,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,1.078342e+10,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,1.021210e+11,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,1.813863e+11,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,9.876586e+10,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [ ]:
fin.isna().sum()

Symbol            0
Name              0
Sector            0
Price             0
Price/Earnings    2
Dividend Yield    0
Earnings/Share    0
52 Week Low       0
52 Week High      0
Market Cap        0
EBITDA            0
Price/Sales       0
Price/Book        8
SEC Filings       0
dtype: int64

In [ ]:
fin = fin.fillna(0)

In [ ]:
fin = fin.assign(HighLow=lambda x: (x['52 Week Low'] - x['52 Week High']) / x['Price'])
fin = fin.drop(['Name', 'SEC Filings', 'Sector', 'Price', '52 Week Low', '52 Week High'], axis=1)
fin.drop(fin[fin["Symbol"]=='OGN'].index, inplace=True)
fin.drop(fin[fin["Symbol"]=='AYI'].index, inplace=True)

fin.head()

,Symbol,Price/Earnings,Dividend Yield,Earnings/Share,Market Cap,EBITDA,Price/Sales,Price/Book,HighLow
0,MMM,24.31,2.332862,7.92,1.387211e+11,9.048000e+09,4.390271,11.34,0.378124
1,AOS,27.76,1.147959,1.70,1.078342e+10,6.010000e+08,3.575483,6.35,0.323124
2,ABT,22.51,1.908982,0.26,1.021210e+11,5.744000e+09,3.740480,3.19,0.396659
3,ABBV,19.41,2.499560,3.29,1.813863e+11,1.031000e+10,6.291571,26.14,0.606656
4,ACN,25.47,1.714470,5.44,9.876586e+10,5.643228e+09,2.604117,10.62,0.317454


In [ ]:
price = pd.read_csv('/content/sp500_stocks.csv')
price.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2009-12-31,MMM,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-04,MMM,59.318886,83.019997,83.449997,82.669998,83.089996,3043700.0
2,2010-01-05,MMM,58.947342,82.500000,83.230003,81.699997,82.800003,2847000.0
3,2010-01-06,MMM,59.783295,83.669998,84.599998,83.510002,83.879997,5268500.0
4,2010-01-07,MMM,59.826176,83.730003,83.760002,82.120003,83.320000,4470100.0


In [ ]:
price = price.drop(['Adj Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
price = price.dropna()
price.drop(price[price["Symbol"]=='OGN'].index, inplace=True)
price['Date'] = price.Date.str[0:7] + ' ' + price['Symbol']
price.drop_duplicates(subset ='Date', keep ='first', inplace =True)
price['Year'] = price.Date.str[:4]
price['Month'] = price.Date.str[5:7]
price = price.drop(['Date'], axis=1)
price = price[(price['Year'] == '2021') | (price['Year'] == '2020')]
price.drop(price[price["Symbol"]=='OGN'].index, inplace=True)
company = price['Symbol'].unique()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


MOMENTUM CALCULATIONS

In [ ]:
monthMove = {}
month3Move = {}
month6Move = {}
month12Move = {}


for i in company:
    
    #
    m4 = price[(price['Symbol'] == i) & (price['Year'] == '2021') & (price['Month'] == '08')]
    m1 = price[(price['Symbol'] == i) & (price['Year'] == '2021') & (price['Month'] == '11')]

    month3Move[i] = []
    month3Move[i].append((m1.iloc[0]['Close']) / (m4.iloc[0]['Close']))
    
    #
    m7 = price[(price['Symbol'] == i) & (price['Year'] == '2021') & (price['Month'] == '05')]
    m1 = price[(price['Symbol'] == i) & (price['Year'] == '2021') & (price['Month'] == '11')]

    month6Move[i] = []
    month6Move[i].append((m1.iloc[0]['Close']) / (m7.iloc[0]['Close']))   
    
    #
    m13 = price[(price['Symbol'] == i) & (price['Year'] == '2020') & (price['Month'] == '11')]
    m1 = price[(price['Symbol'] == i) & (price['Year'] == '2021') & (price['Month'] == '11')]

    month12Move[i] = []
    month12Move[i].append((m1.iloc[0]['Close']) / (m13.iloc[0]['Close'])) 
        
    #
    curr = price[(price['Symbol'] == i) & (price['Year'] == '2021') & (price['Month'] == '12')]
    m1 = price[(price['Symbol'] == i) & (price['Year'] == '2021') & (price['Month'] == '11')]

    monthMove[i] = []
    monthMove[i].append((m1.iloc[0]['Close']) / (curr.iloc[0]['Close']))


In [ ]:
#
dict3 = {}

for i in fin['Symbol']:
    if i not in month3Move:
        pass
    else:
        dict3[i] = []
        dict3[i].append(month3Move[i])

#
dict6 = {}

for i in fin['Symbol']:
    if i not in month6Move:
        pass
    else:
        dict6[i] = []
        dict6[i].append(month6Move[i])

#
dict12 = {}

for i in fin['Symbol']:
    if i not in month12Move:
        pass
    else:
        dict12[i] = []
        dict12[i].append(month12Move[i])

#       
targetdict = {}

for i in fin['Symbol']:
    if i not in monthMove:
        pass
    else:
        targetdict[i] = []
        targetdict[i].append(monthMove[i])

In [ ]:
for i in fin["Symbol"]:
    if i not in targetdict:
        fin.drop(fin[fin["Symbol"]==i].index, inplace=True)

In [ ]:
#
m3 = pd.DataFrame.from_dict(month3Move, orient='index', columns=['3M Momentum']).reset_index()
m3 = m3.rename(columns={'index': 'Symbol'})

#
m6 = pd.DataFrame.from_dict(month6Move, orient='index', columns=['6M Momentum']).reset_index()
m6 = m6.rename(columns={'index': 'Symbol'})

#
m12 = pd.DataFrame.from_dict(month12Move, orient='index', columns=['12M Momentum']).reset_index()
m12 = m12.rename(columns={'index': 'Symbol'})

#
target = pd.DataFrame.from_dict(targetdict, orient='index', columns=['Target']).reset_index()
target = target.rename(columns={'index': 'Symbol'})

In [ ]:
momentum = pd.merge(pd.merge(target,m3, on='Symbol'), m6, on='Symbol')
momentum = pd.merge(momentum, m12, on='Symbol')
momentum['Target'] = momentum.Target.str[0]

momentum.sort_values(by=['Symbol']).head()

,Symbol,Target,3M Momentum,6M Momentum,12M Momentum
11,A,1.059713,1.026066,1.179218,1.492398
26,AAL,1.214373,0.985543,0.900683,1.766756
7,AAP,1.031526,1.058295,1.115179,1.487052
41,AAPL,0.904048,1.023639,1.123887,1.369495
3,ABBV,0.990510,0.994456,1.001134,1.300963


Put the dataframes together

In [ ]:
df = pd.merge(fin, momentum, on='Symbol')
df = df.sort_values(by=['Symbol'])
df.head()

,Symbol,Price/Earnings,Dividend Yield,Earnings/Share,Market Cap,EBITDA,Price/Sales,Price/Book,HighLow,Target,3M Momentum,6M Momentum,12M Momentum
11,A,27.45,0.875698,2.10,2.198461e+10,1.094000e+09,6.493563,4.56,0.396157,1.059713,1.026066,1.179218,1.492398
26,AAL,9.92,0.778210,3.91,2.459485e+10,5.761000e+09,0.580226,6.03,0.408848,1.214373,0.985543,0.900683,1.766756
7,AAP,19.54,0.218321,6.19,8.123612e+09,8.539410e+08,1.130106,2.51,0.827693,1.031526,1.058295,1.115179,1.487052
41,AAPL,16.86,1.579541,9.20,8.095080e+11,7.938600e+10,3.458609,5.66,0.315694,0.904048,1.023639,1.123887,1.369495
3,ABBV,19.41,2.499560,3.29,1.813863e+11,1.031000e+10,6.291571,26.14,0.606656,0.990510,0.994456,1.001134,1.300963


The target will be to predict a stock that will return more than 2% in the next month.
This will equal 25%+ annualized.
This is significantly higher than the index to keep a margin of safety

In [ ]:
df['Target'] = [0 if x <1.02 else 1 for x in df['Target']]
df.head()

,Symbol,Price/Earnings,Dividend Yield,Earnings/Share,Market Cap,EBITDA,Price/Sales,Price/Book,HighLow,Target,3M Momentum,6M Momentum,12M Momentum
11,A,27.45,0.875698,2.10,2.198461e+10,1.094000e+09,6.493563,4.56,0.396157,1,1.026066,1.179218,1.492398
26,AAL,9.92,0.778210,3.91,2.459485e+10,5.761000e+09,0.580226,6.03,0.408848,1,0.985543,0.900683,1.766756
7,AAP,19.54,0.218321,6.19,8.123612e+09,8.539410e+08,1.130106,2.51,0.827693,1,1.058295,1.115179,1.487052
41,AAPL,16.86,1.579541,9.20,8.095080e+11,7.938600e+10,3.458609,5.66,0.315694,0,1.023639,1.123887,1.369495
3,ABBV,19.41,2.499560,3.29,1.813863e+11,1.031000e+10,6.291571,26.14,0.606656,0,0.994456,1.001134,1.300963


In [ ]:
X = df.drop(['Target','Symbol'], axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Models

In [ ]:
models = [SVC(kernel='rbf'),
          RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=100),
          AdaBoostClassifier(n_estimators=100, random_state=101),
          KNeighborsClassifier(n_neighbors=1),
          GaussianNB(),
          XGBClassifier(),
          LogisticRegression(random_state=0),
          Perceptron(tol=1e-3, random_state=0)
          ]

In [ ]:
accuracy = []

for model in models:
    cross_val = cross_val_score(model, X_train, y_train, scoring='accuracy',
                                cv=StratifiedKFold(10)).mean()
    accuracy.append(cross_val)

models_name = ['SVC', 'RandomForestClassifier', 'ADA',
               'KNN', 'GNB', 'XGB', 'LogReg', 'Perceptron']

acc = pd.DataFrame({'Model': models_name, 'Accuracy': accuracy})
acc

,Model,Accuracy
0,SVC,0.703333
1,RandomForestClassifier,0.766667
2,ADA,0.686667
3,KNN,0.723333
4,GNB,0.636667
5,XGB,0.736667
6,LogReg,0.363333
7,Perceptron,0.536667


Simple Neural Network

In [ ]:
model = keras.Sequential([
    layers.Dense(units=256, activation='relu', input_shape=[11]),
    layers.Dense(units=256, activation='relu'),
    layers.Dense(units=1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0,
)

history_df = pd.DataFrame(history.history)

nn = {'Model': 'Neural Network', 'Accuracy': (history_df['val_binary_accuracy'].max())}
acc = acc.append(nn, ignore_index=True)

print(("Best Validation Accuracy: {:0.4f}")\
      .format(history_df['val_binary_accuracy'].max()))

Best Validation Accuracy: 0.7333


In [ ]:
acc

,Model,Accuracy
0,SVC,0.703333
1,RandomForestClassifier,0.766667
2,ADA,0.686667
3,KNN,0.723333
4,GNB,0.636667
5,XGB,0.736667
6,LogReg,0.363333
7,Perceptron,0.536667
8,Neural Network,0.733333


Simple Neural Network with Dropout and BatchNormalization

In [ ]:
model = keras.Sequential([
    layers.Dense(units=1024, activation='relu', input_shape=[11]),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units=1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units=1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units=1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0,
)

history_df = pd.DataFrame(history.history)

nn_dropout = {'Model': 'Neural Network w/ Dropout', 'Accuracy': (history_df['val_binary_accuracy'].max())}
acc = acc.append(nn_dropout, ignore_index=True)

print(("Best Validation Accuracy: {:0.4f}")\
      .format(history_df['val_binary_accuracy'].max()))

Best Validation Accuracy: 0.4000


Value only

In [ ]:
df.head()

,Symbol,Price/Earnings,Dividend Yield,Earnings/Share,Market Cap,EBITDA,Price/Sales,Price/Book,HighLow,Target,3M Momentum,6M Momentum,12M Momentum
11,A,27.45,0.875698,2.10,2.198461e+10,1.094000e+09,6.493563,4.56,0.396157,1,1.026066,1.179218,1.492398
26,AAL,9.92,0.778210,3.91,2.459485e+10,5.761000e+09,0.580226,6.03,0.408848,1,0.985543,0.900683,1.766756
7,AAP,19.54,0.218321,6.19,8.123612e+09,8.539410e+08,1.130106,2.51,0.827693,1,1.058295,1.115179,1.487052
41,AAPL,16.86,1.579541,9.20,8.095080e+11,7.938600e+10,3.458609,5.66,0.315694,0,1.023639,1.123887,1.369495
3,ABBV,19.41,2.499560,3.29,1.813863e+11,1.031000e+10,6.291571,26.14,0.606656,0,0.994456,1.001134,1.300963


In [ ]:
Xv = df.drop(['Target','Symbol', '3M Momentum', '6M Momentum', '12M Momentum'], axis=1)
yv = df['Target']

Xv_train, Xv_test, yv_train, yv_test = train_test_split(Xv, yv, test_size=0.2, random_state=42)

In [ ]:
value_accuracy = []

for model in models:
    cross_val = cross_val_score(model, Xv_train, yv_train, scoring='accuracy',
                                cv=StratifiedKFold(10)).mean()
    value_accuracy.append(cross_val)

models_name = ['SVC', 'RandomForestClassifier', 'ADA',
               'KNN', 'GNB', 'XGB', 'LogReg', 'Perceptron']

value_acc = pd.DataFrame({'Model': models_name, 'Value Accuracy': value_accuracy})
value_acc

,Model,Value Accuracy
0,SVC,0.703333
1,RandomForestClassifier,0.643333
2,ADA,0.626667
3,KNN,0.723333
4,GNB,0.636667
5,XGB,0.656667
6,LogReg,0.363333
7,Perceptron,0.536667


Simple Neural Network

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(units=256, activation='relu', input_shape=[8]),
    layers.Dense(units=256, activation='relu'),
    layers.Dense(units=1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    Xv_train, yv_train,
    validation_data=(Xv_test, yv_test),
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0,
)

history_df = pd.DataFrame(history.history)

value_nn = {'Model': 'Neural Network', 'Value Accuracy': (history_df['val_binary_accuracy'].max())}
value_acc = value_acc.append(value_nn, ignore_index=True)

print(("Best Validation Accuracy: {:0.4f}")\
      .format(history_df['val_binary_accuracy'].max()))

Best Validation Accuracy: 0.7333


Simple Neural Network with Dropout and BatchNormalization¶

In [ ]:
model = keras.Sequential([
    layers.Dense(units=1024, activation='relu', input_shape=[8]),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units=1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units=1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units=1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    Xv_train, yv_train,
    validation_data=(Xv_test, yv_test),
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0,
)

history_df = pd.DataFrame(history.history)

value_nn2 = {'Model': 'Neural Network w/ Dropout', 'Value Accuracy': (history_df['val_binary_accuracy'].max())}
value_acc = value_acc.append(value_nn2, ignore_index=True)

print(("Best Validation Accuracy: {:0.4f}")\
      .format(history_df['val_binary_accuracy'].max()))

Best Validation Accuracy: 0.4000


In [ ]:
df = pd.concat([acc, value_acc['Value Accuracy']], axis=1)

In [ ]:
df

,Model,Accuracy,Value Accuracy
0,SVC,0.703333,0.703333
1,RandomForestClassifier,0.766667,0.643333
2,ADA,0.686667,0.626667
3,KNN,0.723333,0.723333
4,GNB,0.636667,0.636667
5,XGB,0.736667,0.656667
6,LogReg,0.363333,0.363333
7,Perceptron,0.536667,0.536667
8,Neural Network,0.733333,0.733333
9,Neural Network w/ Dropout,0.400000,0.400000


CONCLUSION

Looks like the Simple Neural Network performs best

I used a snapshot of 1 month for the data, price and financial data has been available for decades.

I think the models could be improved further by gathering all these data.

Thanks for reading and all feedback is appreciated.